In [1]:
#from selenium import webdriver
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [ ]:
chrome_driver_loc = 'C:/Users/nikhi/Downloads/chromedriver'
driver = webdriver.Chrome(chrome_driver_loc)

In [2]:
links = pd.read_csv('C:/Users/nikhi/Dropbox/Jharkhand Minerals/Lessee Profile Links.csv')

In [ ]:
#RML - rptAccordian_ctl00_tblView
#AUCTION - rptAccordian_ctl00_tblView

In [ ]:
i = 37
link = links['detail_link'][i]
driver.get(link)

In [3]:
def remove_columns(df):
    cols = list(df.columns)
    for i in range(len(cols)):
        c = cols[i]
        try :
            temp = int(c[c.rfind('.')+1:])
            cols[i] = c[:c.rfind('.')]
        except:
            continue
    return pd.DataFrame([cols]+df.values.tolist())

In [4]:
def get_update_status(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content)
    tabs = soup.find_all("div",{"id":"myTab"})
    images = []
    for tab in tabs:
        images.extend(tab.find_all('img'))
    tags = {}
    count = 1
    for image in images:
        if 'oknew' in str(image):
            tags.update({count:'Updated'})
        if 'cancelnew' in str(image):
            tags.update({count:'Not Updated'})
        count+=1
    return {'updateStatus':tags}

In [5]:
#LESSEE DETAILS
def get_lessee_details(link):
    linkId = 'LesseeProfile'
    tableId = 'grdLProfile'
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    pre = ''
    details = {}
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[1]
    except:
        return details
    for l in df.values:
        if len(set([x for x in l if x==x])) == 1:
            pre = l[0] +' '
            continue
        i=1
        last_detail = 'abcdef'
        while i<len(l):
            if l[i-1]==last_detail:
                i+=1
                continue
            details.update({pre+str(l[i-1]):l[i]})
            last_detail = l[i]
            if l[i] == 'Contact Person Name':
                break
            i+=2
    return {linkId:details}

In [6]:
#LEASE INFO
def get_lease_info(link):
    linkId = 'LesseeInfo'
    tableId = 'grdLeaseInfo'
    pre = ''
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        df = pd.read_html(str(soup.find("table",{'id':tableId})))[1]
    except:
        return details
    for l in df.values:
        if len(set([x for x in l if x==x])) == 1:
            pre = l[0] +' '
            continue
        i=1
        if l[0] == 'Non Forest Area (HA)':
            pre = pre+l[0]+' '
            i=2
        last_detail = 'abcdef'
        while i<len(l):
            if l[i-1]==last_detail:
                i+=1
                continue
            details.update({pre+str(l[i-1]):l[i]})
            last_detail = l[i]
            i+=2
    return {linkId:details}

In [7]:
#FOREST CLEARANCE
def get_forest_clearance(link):
    linkId = 'ForestClearance'
    tableId = 'grdForClear'
    pre = ''
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[1]
    except:
        return details
    df = df['Forest Clearance']
    df = remove_columns(df)
    for l in df.values:
        if len(set([x for x in l if x==x])) == 1:
            pre = l[0] +' '
            continue
        i=1
        if l[0] == 'Stage 1 Clearance':
            pre = pre+l[0]+' '
            i=3
        elif l[0] == 'Stage 2 Clearance':
            pre = 'Non-Forest Land Details '+l[0]+' '
            i=3        
        last_detail = 'abcdef'
        while i<len(l):
            if l[i-1]==last_detail:
                i+=1
                continue
            details.update({pre+str(l[i-1]):l[i]})
            last_detail = l[i]
            i+=2
    return {linkId:details}

In [8]:
#ENVIRONMENT CLEARANCE
def get_env_clearance(link):
    linkId = 'EnvClearance'
    tableId = 'grdEnv'
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    details = {}
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[0]
    except:
        return details
    for i in df.index:
        for c in df.columns:
            details.update({c+' '+str(i+1):df[c][i]})
    return {linkId:details}

In [9]:
#POLLUTION CLEARANCE
def get_pol_clearance(link):
    linkId = 'PCBClearance'
    tableId = 'grdOspcbML'
    pre = ''
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        dfs = pd.read_html(str(soup.find("table",{"id":tableId})))
    except:
        return details
    for j in range(1,len(dfs)):
        df = remove_columns(dfs[j])
        for l in df.values:
            if len(set([x for x in l if x==x])) == 1:
                pre = l[0] +' ' + str(j)+' '
                continue
            i=1
            last_detail = 'abcdef'
            while i<len(l):
                if l[i-1]==last_detail:
                    i+=1
                    continue
                details.update({pre+str(l[i-1]):l[i]})
                last_detail = l[i]
                i+=2
    return {linkId:details}

In [10]:
#SURVEY
def get_survey_details(link):
    linkId = 'Survey'
    tableId = 'grdSurveyML'
    pre = ''
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[1]
    except:
        return details
    df = remove_columns(df)
    for l in df.values:
        if len(set([x for x in l if x==x])) == 1:
            pre = l[0] +' '
            continue
        i=1
        last_detail = 'abcdef'
        while i<len(l):
            if l[i-1]==last_detail:
                i+=1
                continue
            details.update({pre+str(l[i-1]):l[i]})
            last_detail = l[i]
            i+=2
    return {linkId:details}

In [11]:
#MINING PLAN
def get_mining_plan_details(link):
    linkId = 'MiningPlan'
    tableId = 'tbML'
    pre = 'Mining Plan '
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[0].iloc[1:5,1:]
    except:
        return details
    for l in df.values:
        if len(set([x for x in l if x==x])) == 1:
            pre = l[0] +' '
            continue
        i=1
        last_detail = 'abcdef'
        while i<len(l):
            if l[i-1]==last_detail:
                i+=1
                continue
            if pre in l[i-1]:
                details.update({l[i-1]:l[i]})
            else:
                details.update({pre+str(l[i-1]):l[i]})
            last_detail = l[i]
            i+=2
    return {linkId+'Details':details}

In [12]:
#MINING PLAN 2
def get_mining_plan_grid_details(link):
    linkId = 'MiningPlan'
    tableId = 'grdMiningPlan'
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    details = {}
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[0]
    except:
        return details
    for i in df.index:
        for c in df.columns:
            details.update({c+' '+str(i+1):df[c][i]})
    return {linkId:details}

In [13]:
#GRANT/EXECUTION
def get_grant_details(link):
    linkId = 'Grant'
    tableId = 'grLGrant'
    pre = ''
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[1]
    except:
        return details
    df = remove_columns(df)
    for l in df.values:
        if len(set([x for x in l if x==x])) == 1:
            pre = l[0] +' '
            continue
        i=1
        last_detail = 'abcdef'
        while i<len(l):
            if l[i-1]==last_detail:
                i+=1
                continue
            details.update({pre+str(l[i-1]):l[i]})
            last_detail = l[i]
            i+=2
    return {linkId:details}

In [14]:
#SURFACE RIGHT GRANT/POSSESSION
def get_surface_right_details(link):
    linkId = 'SurfaceRightCL'
    tableId = 'grdSurface'
    pre = ''
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        dfs = pd.read_html(str(soup.find("table",{"id":tableId})))
    except:
        return details
    for j in range(1,3):
        df = remove_columns(dfs[j])
        for l in df.values:
            if len(set([x for x in l if x==x])) == 1:
                pre = l[0] +' '
                continue
            i=1
            last_detail = 'abcdef'
            while i<len(l):
                if l[i-1]==last_detail:
                    i+=1
                    continue
                details.update({pre+str(l[i-1]):l[i]})
                last_detail = l[i]
                if '(HA)' in l[i-1]:
                    break
                i+=2
    return {linkId:details}

In [15]:
#DCC
def get_dcc_details(link):
    linkId = 'DCC'
    tableId = 'grdDCC'
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[0]
    except:
        return details
    for i in df.index:
        for c in df.columns:
            details.update({c+' '+str(i+1):df[c][i]})
    return {linkId:details}

In [20]:
mainDict = {}
for i in links[links['ML/RML']=='ML'].index:
    try:
        link = links['detail_link'][i]
        code = links['Lessee Code'][i]
        detailsDict = {}
        detailsDict.update(get_lessee_details(link))
        detailsDict.update(get_lease_info(link))
        detailsDict.update(get_forest_clearance(link))
        detailsDict.update(get_env_clearance(link))
        detailsDict.update(get_pol_clearance(link))
        detailsDict.update(get_survey_details(link))
        detailsDict.update(get_mining_plan_details(link))
        detailsDict.update(get_mining_plan_grid_details(link))
        detailsDict.update(get_grant_details(link))
        detailsDict.update(get_surface_right_details(link))
        detailsDict.update(get_dcc_details(link))
        detailsDict.update(get_update_status(link))
        mainDict.update({int(code):detailsDict})
    except:
        continue
np.save("C:/Users/nikhi/Dropbox/Jharkhand Minerals/newDetailsDict.npy",mainDict)